<a href="https://colab.research.google.com/github/GouthamVicky/LLM-LongDoc-Summary/blob/main/FalconSummaryEvaluvation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Evaluvation

## Install the required Libraries

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install -U sentence-transformers
!pip install rouge_score
!pip install tqdm

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134

# Download data
 Dataset used - https://huggingface.co/datasets/ccdv/arxiv-summarization

In [ ]:
from datasets import load_dataset
from random import randrange
import re
from random import randrange
import warnings


dataset_name = "ccdv/arxiv-summarization"

#Load Test data from Hugginface dataset
dataset = load_dataset(dataset_name, split='test[:1%]')


dataset=dataset.select([i for i in range(50)])

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])



Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

dataset size: 50
{'article': 'entanglement @xcite in a composite system refers to certain implicit correlation between the subsystems arising from their interaction . \n it is the key resource of quantum computation and quantum information processing @xcite . \n due to recent advances in this field , entanglement has generated renewed interest . \n there have been different approaches to understand and to quantify entanglement @xcite . \n but so far the entanglement , only in a bipartite pure state has been investigated very extensively . \n the von neumann entropy @xcite of either of the subsystems provides a good measure of entanglement in this case @xcite . \n this is the quantum partner of the shannon s entropy @xcite in classical information theory and is defined as @xcite @xmath1 where @xmath2 . here , @xmath3 is the reduced density operator of the subsystem @xmath4 and is given by @xmath5 where @xmath6 is the density operator of the composite system under consideration and @xmat

## Import libraries

In [ ]:

import nltk
import pandas as pd
from nltk.cluster import KMeansClusterer
import numpy as np

import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial import distance
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer,util

#Download Sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

nltk.download('punkt')
nltk.download('stopwords')

# Ignore all warnings

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Preprocessing pipeline

- Clean and tokenize sentences
- Generate embedding representation using sentence-transformer
- Create a cluster using k-means
- Finding the nearest cluster from centroid
- Return the extractive summary passage

In [ ]:
# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

def clean_and_tokenize(sentence):
    sentence = re.sub('[^a-zA-Z0-9.]', ' ', sentence)
    sentence = sentence.lower()
    tokens = sentence.split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

def extractive_summary_generator(input_article_text):
    sentences = sent_tokenize(input_article_text)
    corpus = [clean_and_tokenize(sentence) for sentence in sentences]

    # Encode sentences using Sentence Transformers
    sentence_embeddings = model.encode(corpus)

    n_clusters = 20
    if len(sentence_embeddings) >= n_clusters:
        kmeans = KMeans(n_clusters, init='k-means++', random_state=42)
        y_kmeans = kmeans.fit_predict(sentence_embeddings)

        # Finding and printing the nearest sentence vector from cluster centroid
        my_list = []
        for i in range(n_clusters):
            my_dict = {}

            for j in range(len(y_kmeans)):
                if y_kmeans[j] == i:
                    my_dict[j] = distance.euclidean(kmeans.cluster_centers_[i], sentence_embeddings[j])

            if my_dict:  # Check if my_dict is not empty
                min_distance = min(my_dict.values())
                my_list.append(min(my_dict, key=my_dict.get))


        sentences = [sentences[i] for i in sorted(my_list) if len(sentences[i].split()) >= 4]
        extractive_summary = "\n".join(sentences)
        return extractive_summary
    else:
        return "\n".join(corpus)


def preprocess_text(text):
    #Remove special characters and extra whitespace
    text = re.sub(r'[^a-zA-Z0-9.\s]', '', text)
    text = ' '.join(text.split())

    # Remove extra spaces
    text = ' '.join(text.split())

    # Remove extra multi-line breaks
    text = re.sub(r'\n\s*\n', '\n\n', text)

    return extractive_summary_generator(text)


def remove_pattern_words(text):
    # Define patterns to match 'xmath' and 'xcite' and their variations
    pattern_xmath = r'xmath\d+'
    pattern_xcite = r'xcite'

    # Replace the pattern words with an empty string
    text_without_pattern_words = re.sub(pattern_xmath, '', text)
    text_without_pattern_words = re.sub(pattern_xcite, '', text_without_pattern_words)

    return text_without_pattern_words


def generate_prompt(arxiv_dataset):
    arxiv_dataset['prompts'] = remove_pattern_words(f"### Please give me a brief summary of this research paper\n" \
                              f"### Paper : {preprocess_text(str(arxiv_dataset['article']))}\n\n" \
                              f"### Summary : {str(arxiv_dataset['abstract'])}" if arxiv_dataset.get("abstract") else None)
    return arxiv_dataset

# Assuming you have a 'dataset' object, apply the transformations
dataset = dataset.map(generate_prompt, remove_columns=list(dataset.features))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# Check the Prompt format

from random import randint

print(dataset[randint(0, len(dataset))]["prompts"])

### Please give me a brief summary of this research paper
### Paper : shown to be spatially coincident with the sn in pre sn images with roughly two dozen upper limits derived from non detections  .
this review is organized as follows .
following a brief summary of sn classification and stellar evolution theory 2 one example from each of the following three categories of progenitor studies is provided 3 ordered from most to least common 1 no progenitor star detected in pre sn images 2 likely progenitor star identified via spatial coincidence in pre sn and post sn images 3 progenitor star detected in pre sn images and subsequently confirmed by demonstrating its absence in images taken after the sn has faded beyond detection .
 see also the article by elias rosa in this volume and sn progenitor forensics e.g.
for a comprehensive discussion of all such related areas see the recent review by  .
ii p are the least stripped at the time of explosion and ib c are the most stripped .
among the 

In [ ]:
sample_text="""### Please give me a brief summary of this research paper
### Paper : why does dark energy become dominant quite late in the history of the universe models inspired by string theory like 4form gauge models can describe the smallness of the dark energy but not the coincidence problem .
anthropical models explain the latter problem but it is very difficult to find a natural and convincing particle physics model for them .
the same problem somehow exists for the alternative to a cosmological constant i.e.
for quintessence models .
even if tracker solutions make the model not very sensitive to the initial conditions some fine tuning of the slope of the potential is necessary .
here we suggest an alternative to a primordial quintessence field .
there are at least two motivations for the existence of a decaying dark matter ddm .
if r parity in susy models is not strictly conserved the lsp which is one of the best candidates of dm can decay to standard model particles .
violation of this symmetry is one of the many ways for providing neutrinos with very small mass and large mixing angle .
another motivation is the search for sources of ultra high energy cosmic rays uhecrs .
in this case ddm must be composed of ultra heavy particles with  .
in a recent work we have shown that the lifetime of uhdm ultra heavy dark matter can be relatively short i.e.
 where  is the age of the universe astro ph0001137 .
if a very small fraction of the mass of primary ddm particles changes to a scalar field with proper self interaction potential the gradual condensation of this field at late time behaves like a quintessential matter .
in the present work we only study the plausibility of this model .
we postpone a more detailed study to elsewhere .
the natural choice of cosmological parameters for this model is an initial  where  is an early time in the history of the universe .
for the result presented here we consider it to be the time of decoupling of cmb photons  .
there is strict constraint on the amount of the latter from cosmological observations and the model must be consistent with observations .
however one should not forget that massive particles like proton ant proton and even electrons become colder with the expansion of the universe and at some point they are not any more considered as hot .
we summarize our preliminary results in two following figures .
the first figure shows  of the fit of quintessence models on the publicly available super novae ia data .
models with  fit the data better than  .
in one hand it shows that it is possible to obtain the present equivalent value of cosmological parameters without fine tuning of the suggested model .
the measurement of cosmological parameters at high redshift permits to distinguish between this model and other quintessence models .
it is evident that the model presented here can not be believed before investigating many issues .
the first and one of the most important ones is the condensation of the scalar field .
the other issue is that the value of  for this type of matter can not be constant .
this can affect the evolution of halos star formation rate ionization of igm etc .
and can be used to verify the model .

### Summary : """


## Model Download

- Download the model from Huggingface model card
- Initiate the huggingface pipeline to load model and tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "GouthamVignesh/falcon-arxiv-long-summary-1B"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


### Generate a summary for sample text

In [ ]:
results = " ".join([f"{seq['generated_text']}" for seq in pipeline(sample_text, max_length=1024, do_sample=True, top_k=10, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)])
generated_summary=results.split("### Summary :")[-1]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_summary

' \n### What is the relationship between the number of primordial dark matter nuclei and the mass of the ddm?\nIn the following figure the value of is shown as a function of the age of the universe.\nwe consider the value of.\nthe mass function in the figure is the same for both the models.\nif we have a mass function that is a constant of the universe we can use it to obtain the mass of primordial dark matter.\nthe values of for this model in the table below show that the value of is not a constant.\nwe show below that the value of the mass function is constant and therefore we can not say for sure that the value of the mass of primordial dark matter will be a constant.\nthe mass function shows that the value of the mass function changes from early times.\nhowever there exist several cases.\nif we consider the mass function and the value of.\nwe can see that the value of the mass function is the same in all cases.\nso we can say that the value of the mass function does not change at h

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Model evaluvaton

-  Load Dataset and Initialize Lists.
-  Initialize Sentence Transformer Model.
-  Calculate BLEU, ROUGE, and Semantic Similarity Scores.
-  Iterate and Evaluate Each Example.
-  Calculate Average BLEU, ROUGE, and Semantic Similarity Scores.
-  Combine Scores into a Hybrid Score.
-  Interpret the Results.

In [ ]:
from tqdm import tqdm  # Import tqdm
# Initialize lists to store evaluation scores

sentence_transformer_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
bleu_scores = []
rouge_scores = []
semantic_similarity_scores = []

evaluation_dataset = dataset.select([i for i in range(50)])
# Use tqdm to iterate through the dataset and generate/evaluate summaries
for example in tqdm(evaluation_dataset, desc="Processing"):
    prompt = example['prompts']
    reference_summary = prompt.split("### Summary :")[-1]

    # Generate a summary based on the prompt
    results = " ".join([f"{seq['generated_text']}" for seq in pipeline(sample_text, max_length=1024, do_sample=True, top_k=10, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)])
    generated_summary=results.split("### Summary :")[-1]

    # Calculate BLEU score for the generated summary
    bleu = sentence_bleu([reference_summary.split()], generated_summary.split())
    bleu_scores.append(bleu)

    # Calculate ROUGE score for the generated summary
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
    rouge = scorer.score(reference_summary, generated_summary)
    rouge_scores.append(rouge)

    # Calculate semantic similarity score using Sentence Transformers
    reference_embedding = sentence_transformer_model.encode(reference_summary, convert_to_tensor=True)
    generated_embedding = sentence_transformer_model.encode(generated_summary, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(reference_embedding, generated_embedding)
    semantic_similarity_scores.append(similarity.item())

# Calculate the average scores for each metric
average_bleu = sum(bleu_scores) / len(bleu_scores)
average_rouge = np.mean([rouge['rougeL'].fmeasure for rouge in rouge_scores])
average_semantic_similarity = np.mean(semantic_similarity_scores)

# Combine the scores using a weighted average or other method to create a hybrid score

# Adjust the weights as needed for your specific task
hybrid_score = 0.4 * average_bleu + 0.4 * average_rouge + 0.2 * average_semantic_similarity



Processing: 100%|██████████| 50/50 [05:18<00:00,  6.38s/it]


In [ ]:
print(f"\nAverage BLEU Score: {average_bleu}")
print(f"Average ROUGE Score: {average_rouge}")
print(f"Average Semantic Similarity Score: {average_semantic_similarity}")
print(f"Hybrid Score: {hybrid_score}")


Average BLEU Score: 0.0011057700167721716
Average ROUGE Score: 0.09498982102316
Average Semantic Similarity Score: 0.11340428526513278
Hybrid Score: 0.061119093468999436
